In [81]:
%run downoald_and_cleaning_data.ipynb

Downloading...
From (original): https://drive.google.com/uc?id=1UR_MC8Q5K4JSRnSUX5MQ-sIGPjLLnm9f
From (redirected): https://drive.google.com/uc?id=1UR_MC8Q5K4JSRnSUX5MQ-sIGPjLLnm9f&confirm=t&uuid=fa7f5903-794b-416d-8eaa-178729012eac
To: c:\Users\Anais\Visual studio\PSSD\Data\data.csv
100%|██████████| 363M/363M [00:20<00:00, 17.6MB/s] 


In [82]:
df = pd.read_csv("data_clean.csv")

In [83]:
df.dropna(subset=['texte'], inplace=True)

In [84]:
df['texte'] = df['texte'].str.lower()

In [96]:
import re
from tqdm import tqdm

tqdm.pandas()
df['texte_clean'] = df['texte'].str.lower().progress_apply(lambda x: re.sub(r'[^\w\s]', '', x))
df['titre_clean'] = df['titre'].str.lower().progress_apply(lambda x: re.sub(r'[^\w\s]', '', x))


100%|██████████| 81557/81557 [00:00<00:00, 208843.03it/s]


In [99]:

from transformers import pipeline
classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli")
candidate_labels = [
    "fait divers",
    "affaire criminelle",
    "actualité judiciaire",
    "analyse politique",
    "actualité économique",
    "culture",
    "environnement"
]
def batch_classify(df, text_column='texte', batch_size=16):
    themes = []
    texts = df[text_column].tolist()
    for i in tqdm(range(0, len(texts), batch_size)):
        batch = texts[i:i+batch_size]
        batch_valid = [text if isinstance(text, str) and text.strip() != "" else " " for text in batch]
        results = classifier(batch_valid, candidate_labels)
        if isinstance(results, list):
            for res in results:
                themes.append(res['labels'][0])
        else:
            themes.append(results['labels'][0])
    return themes




Device set to use cpu


In [89]:
random_article = df.sample()
print(random_article)


       Unnamed: 0  journal_clean  \
73626       73626  FranceTv Info   

                                                   titre  annee  mois  jour  \
73626  MeToo à l'hôpital : "Beaucoup de sous-entendus...   2024     5     6   

                                                   texte  \
73626  metoo à l'hôpital : "beaucoup de sous-entendus...   

                                                keywords  \
73626  sous-entendus, soignante, ancienne_soignante, ...   

                                             texte_clean  \
73626  metoo à lhôpital  beaucoup de sousentendus et ...   

                                                  tokens  
73626  [metoo, à, lhôpital, beaucoup, de, sousentendu...  


In [103]:
df_random = df.sample(1000)

df_random = df_random.reset_index(drop=True)
df_random['theme'] = batch_classify(df_random, text_column='titre_clean', batch_size=16)

100%|██████████| 63/63 [22:18<00:00, 21.25s/it]


In [104]:
df_random

,Unnamed: 0,journal_clean,titre,annee,mois,jour,texte,keywords,texte_clean,tokens,titre_clean,theme
0,50452,Valeurs Actuelles,Cinéma Les Choses humaines d'Yvan Attal,2021,12,2,"ambigu brillant étudiant à stanford, alexandre...","alexandre, attal, yvan",ambigu brillant étudiant à stanford alexandre ...,"[ambigu, brillant, étudiant, à, stanford, alex...",cinéma les choses humaines dyvan attal,culture
1,52843,Aujourd'hui en France,"«Picard», «Alerte rouge», «Un bail en enfer».....",2022,3,11,"comme chaque week-end, le parisien vous propos...","ligne, dernière, pixar, alerte, studios, semai...",comme chaque weekend le parisien vous propose ...,"[comme, chaque, weekend, le, parisien, vous, p...",picard alerte rouge un bail en enfer 5 nouveau...,fait divers
2,79008,Maville,Ces faits divers oubliés qui ont secoué l’oues...,2024,9,22,info redon ces faits divers oubliés qui ont se...,NaN,info redon ces faits divers oubliés qui ont se...,"[info, redon, ces, faits, divers, oubliés, qui...",ces faits divers oubliés qui ont secoué louest...,fait divers
3,13033,L'Obs,"""Human"" : rien n'est trop beau ni trop grand p...",2015,9,18,toute la planète a été invitée. des diplomates...,"ki-moon, roi, onu, planète, arthus-bertrand, f...",toute la planète a été invitée des diplomates ...,"[toute, la, planète, a, été, invitée, des, dip...",human rien nest trop beau ni trop grand pour ...,culture
4,4899,Ouest-France,"Après l'alcool, la drogue plus facilement dépi...",2008,12,12,"premiers tests salivaires. jeudi après-midi, a...","drogue, salivaires, contrôlé, tests, conducteu...",premiers tests salivaires jeudi aprèsmidi aven...,"[premiers, tests, salivaires, jeudi, aprèsmidi...",après lalcool la drogue plus facilement dépistée,affaire criminelle
...,...,...,...,...,...,...,...,...,...,...,...,...
995,15632,La Dépêche du Midi,"Daniel Toscan du Plantier, une vie de cinéma",2017,1,22,daniel toscan du plantier (1941-2003) a consac...,"plantier, toscan, vie, cinéma, daniel",daniel toscan du plantier 19412003 a consacré ...,"[daniel, toscan, du, plantier, 19412003, a, co...",daniel toscan du plantier une vie de cinéma,environnement
996,25565,Le Télégramme,«#NousToutes ».En route avec le Planning familial,2018,11,22,un appel à se rassembler partout en france est...,"violences, rassembler, Planning_familial, asso...",un appel à se rassembler partout en france est...,"[un, appel, à, se, rassembler, partout, en, fr...",noustoutes en route avec le planning familial,culture
997,7051,Le Berry républicain,"Pigalle, du rock jusqu'au bout",2011,7,5,concert. ils étaient nombreux à venir voir les...,"rock, concert",concert ils étaient nombreux à venir voir les ...,"[concert, ils, étaient, nombreux, à, venir, vo...",pigalle du rock jusquau bout,environnement
998,32085,Madame Figaro,"Beth Ditto, Angèle et Aya Nakamura électrisent...",2019,9,25,21h30. les invités trépignent d'impatience dan...,"beth, défilé, angèle, aya, court, nakamura",21h30 les invités trépignent dimpatience dans ...,"[21h30, les, invités, trépignent, dimpatience,...",beth ditto angèle et aya nakamura électrisent ...,fait divers


In [109]:
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")


c:\Users\Anais\miniconda3\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Anais\.cache\huggingface\hub\models--facebook--bart-large-mnli. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Device set to use cpu


In [110]:
df['theme'] = batch_classify(df, text_column='titre_clean', batch_size=16)

  0%|          | 0/5098 [00:00<?, ?it/s]

  0%|          | 3/5098 [02:26<68:58:58, 48.74s/it]


KeyboardInterrupt: 